In [ ]:
import warnings

# OTE-01: Initial Image Mosaic

In [ ]:
# Standard Library Imports
import os
import re
import time
import multiprocessing

# Third Party Imports
import pprint
from glob import glob
import numpy as np
import webbpsf
from astropy.io import fits
from astropy.io import ascii as asc
import matplotlib.pyplot as plt

# Local Imports (from nircam_simulator package)
from mirage import imaging_simulator
from mirage.seed_image import catalog_seed_image
from mirage.dark import dark_prep
from mirage.ramp_generator import obs_generator
from mirage.apt import apt_inputs
from mirage.yaml import yaml_generator, write_observationlist
from mirage.catalogs import get_catalog
from mirage.psf import psf_library

# View matplotlib plots inline
%matplotlib inline

### Define location of input and output files

In [ ]:
# Where the pointing and XML file for this particular OTE CAR are located
ote_dir = '/user/lchambers/OTECommSims/OTE01_reducedmosaic/'

# Where the output FITS files will be saved to
library_root_dir = '/user/lchambers/OTECommSims/test_library_output/'

In [ ]:
prop_id = 1134

# Change if you named your files differently.
root = 'OTE01-{}-reduced_mosaic'.format(prop_id)

pointing_file = os.path.join(ote_dir, '{}.pointing'.format(root))
xml_file = os.path.join(ote_dir, '{}.xml'.format(root))

---
# First, generate PSF library files

### Get the source catalogs

In [ ]:
# Get SW and LW catalogs
cats = get_catalog.get_all_catalogs(pointing_file, prop_id)
target_coords, catalog_filenames_sw, catalog_filenames_lw = cats

### Get the observation/visit information from the APT files

In [ ]:
# Get the information from the pointing file
apt_prop = apt_inputs.AptInput()
pointing_tab = apt_prop.get_pointing_info(pointing_file, '1140')
n_exposures = len(pointing_tab['visit_id'])

In [ ]:
# Create dictionary that mirrors the program structure
program_structure = {}
for i in range(n_exposures):
    obs_num = pointing_tab['obs_num'][i]
    visit_num = pointing_tab['visit_num'][i]
    activity_id = pointing_tab['act_id'][i]
    
    obs_key = 'Observation{}'.format(obs_num)
    visit_key = 'Visit{}'.format(visit_num)
    
    program_structure.setdefault(obs_key, {})
    visit_dict = program_structure[obs_key].setdefault(visit_key, []).append('Activity{}'.format(activity_id))                                                                      
    
pprint.pprint(program_structure)  

In [ ]:
# Create directory structure based on dictionary

psf_paths = []
program_dir = os.path.join(library_root_dir, root)
for observation in program_structure.keys():
    for visit in program_structure[observation].keys():
        for activity in program_structure[observation][visit]:
            activity_dir = os.path.join(program_dir, observation, visit, activity)
            if not os.path.exists(activity_dir):
                os.makedirs(activity_dir)
            psf_paths.append(activity_dir)

## Generate the PSF array

In [ ]:
nc = webbpsf.NIRCam()
nc, ote = webbpsf.enable_adjustable_ote(nc)

# Add random pistons
random_pistons = np.random.randn(18)*5000 # massive ~cm errors (50000 micron = 5e-2 m)
for i, seg in enumerate(ote.segnames[0:18]):  # don't piston "segment 19" the SM
    ote.move_seg_local(seg, piston=random_pistons[i])
    
# # Just show one segment
# pupil = webbpsf.webbpsf_core.one_segment_pupil('A3')
# ote.amplitude = pupil[0].data

In [ ]:
ote01_psf = nc.calc_psf(monochromatic=2e-6, oversample=1, fov_pixels=2048, display=True)

In [ ]:
webbpsf.display_psf(ote01_psf, vmin=1e-10, vmax=1e-5)

In [ ]:
# Create observation table file from APT files
observationlist_file = os.path.join(os.getcwd(), ote_dir, root + '_observationlist.yaml')
write_observationlist.write_yaml(xml_file, pointing_file, observationlist_file, 
                                 ps_cat_sw=catalog_filenames_sw, ps_cat_lw=catalog_filenames_lw)

In [ ]:
# Create a series of data simulator input yaml files from APT files
yam = yaml_generator.SimInput()

yam.input_xml = xml_file
yam.pointing_file = pointing_file
siaf_file = os.path.expandvars('$MIRAGE_DATA/nircam/reference_files/SIAF/NIRCam_SIAF_2018-01-08.csv')
yam.siaf = siaf_file
yam.output_dir = ote_dir #os.path.join(os.getcwd(), ote_dir)
yam.simdata_output_dir = ote_dir #os.path.join(os.getcwd(), ote_dir)
yam.observation_table = observationlist_file

yam.use_JWST_pipeline = False # changed to False
yam.use_linearized_darks = True # changed to True
yam.datatype = 'linear'

yam.reffile_setup()
yam.create_inputs()

In [ ]:
# Print information about the yaml files that were generated.
yfiles = glob(os.path.join(ote_dir, 'jw*yaml'))

obs_numbers = [f.split('/')[-1].split('_')[0] for f in yfiles]
all_obs_numbers = list(set(obs_numbers))
all_obs_numbers.sort()

n_obs = len(set([int(number[9:11]) for number in all_obs_numbers]))

print('Found {} yaml files.'.format(len(obs_numbers)))
print('({} exposures across {} observations)'.format(len(all_obs_numbers), n_obs))
# pprint.pprint(all_obs_numbers)

In [ ]:
# Get list of YAML files for observation 1 (all 10 detectors)
obs1_visit1_files = glob(os.path.join(ote_dir, 'jw01134001*yaml'))
print('{} yaml files found in program APT 1134 Observation 1'.format(len(obs1_visit1_files)))

In [ ]:
with fits.open(obs1_fits_files[0]) as hdu:
    hdu.info()

In [ ]:
# Generate "mosaic" from images
obs1_fits_files = glob('/user/lchambers/OTECommSims/OTE01_reducedmosaic/jw*linear.fits')

for f in sorted(obs1_fits_files):
    print(f.split('/')[-1])
    with fits.open(f) as hdu:
        print(hdu[0].header['TARG_RA'])
        print(hdu[0].header['RA_V1'])
        print(hdu[0].header['XOFFSET'], hdu[0].header['YOFFSET'])
        print()

---
# Latest version of script:
```python
import datetime

from astropy.io import fits
import webbpsf
import numpy as np

# Generate a PSF with a perturbed OTE according to the expected deployment tolerances.
# Deployment tolerances taken from JWST WFS&C Commissioning and Operations Plan (OTE-24):
# D36168 / 2299462 Rev C Page 10

nc = webbpsf.NIRCam()
nc, ote = webbpsf.enable_adjustable_ote(nc)

# 1. Generate the OPD
# ------------------------------------------------------------------------------

# *** QUESTION: Do I want to pull these random values from a normal distribution? ***


# Add SM moves
random_sm_piston = np.random.random(1) * 5000 - 2500  # microns
random_sm_tilt = np.random.random(2) * 2600 - 1300  # microradians
random_sm_decenter = np.random.random(2) * 5000 - 2500  # microns

ote.move_sm_local(piston=random_sm_piston, xtilt=random_sm_tilt[0],
                  ytilt=random_sm_tilt[1], xtrans=random_sm_decenter[0],
                  ytrans=random_sm_decenter[1])

# Add PMSA (segment) moves
random_pm_piston = np.random.random(18) * 3000 - 1500  # microns
random_pm_tilt = np.random.random((18, 2)) * 2200 - 1100  # microradians
random_pm_decenter = np.random.random((18, 2)) * 2600 - 1300  # microns
random_pm_roc = np.random.random(18) * 302 - 151  # microns
random_pm_clocking = np.random.random(18) * 2400 - 1200  # microradians

global_pm_piston = np.random.random(1) * 1400 - 700  # microns
global_pm_tilt = np.random.random(2) * 380 - 190  # microradians
global_pm_decenter = np.random.random(2) * 400 - 200  # microns
global_pm_clocking = np.random.random(1) * 300 - 150  # microradians

for i, seg in enumerate(ote.segnames[0:18]):
    ote.move_seg_local(seg,
                       piston=random_pm_piston[i] + global_pm_piston,
                       xtilt=random_pm_tilt[i][0] + global_pm_tilt[0],
                       ytilt=random_pm_tilt[i][1] + global_pm_tilt[1],
                       xtrans=random_pm_decenter[i][0] + global_pm_decenter[0],
                       ytrans=random_pm_decenter[i][1] + global_pm_decenter[1],
                       roc=random_pm_roc[i],
                       clocking=random_pm_clocking[i] + global_pm_clocking)

# Save the OPD as a fits file
now = datetime.datetime.now()
ote_filename = "{:2d}{:2d}{:2d}-{:2d}{:2d}{:2d}-OTE01_perturbedOTE.fits".format(
    now.year, now.month, now.day, now.hour, now.minute, now.second
)
hdulist = fits.HDUList(hdus=ote.as_fits())
hdulist.writeto(ote_filename, output_verify='ignore')
print('Save perturbed OTE to {}'.format(ote_filename))

# 2. Generate the PSF for entire OTE pupil
# ------------------------------------------------------------------------------

# Calculate the PSF over 10 x 10 NIRCam fields of view (oy vey)
# Simulating the "Central" observation from APT Program 1134
now = datetime.datetime.now()
psf_filename = "{:2d}{:2d}{:2d}-{:2d}{:2d}{:2d}-OTE01_PSF.fits".format(
    now.year, now.month, now.day, now.hour, now.minute, now.second
)
fov_pixels = 2048 * 10
nc.calc_psf(nlambda=30, oversample=1, fov_pixels=fov_pixels, add_distortion=False,
            outfile=psf_filename)
print('Done generating the PSF!! Oh my goodness!!!')

# 3. Generate the PSF for a single segment pupil... 18 times
# ------------------------------------------------------------------------------

# Then, once all that is done, generate the same thing... for all 18 segments.
# Oy vey x 18
for i_segment in np.arange(1, 19):
    # Restrict the pupil to just be one segment
    pupil = webbpsf.webbpsf_core.one_segment_pupil(i_segment)
    ote.amplitude = pupil[0].data

    # Calculate the PSF over 10 x 10 NIRCam fields of view (oy vey)
    # Simulating the "Central" observation from APT Program 1134
    now = datetime.datetime.now()
    psf_filename = "{:2d}{:2d}{:2d}-{:2d}{:2d}{:2d}-OTE01_segment{}_PSF.fits".format(
        now.year, now.month, now.day, now.hour, now.minute, now.second, i_segment
    )
    fov_pixels = 2048 * 10
    nc.calc_psf(nlambda=30, oversample=1, fov_pixels=fov_pixels, add_distortion=False,
                outfile=psf_filename)
    print('Done generating the PSF for the {}th segment!! Truly a miracle.'.format(i_segment))

print('You will never get here, but I finished. Tada!')
```